In [1]:
from Dataload import dataload
import torch.nn.functional as F

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils

from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
from module import video_model
from constant import EMOTIPATH,EMOTIFACEPATH,EMOTIAUDIOPATH
import os
import pandas as pd
import time
from tqdm.notebook import tqdm
import cv2

from torchvision import transforms


In [2]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Audio_Train=os.path.join(EMOTIAUDIOPATH,"Train_audio.csv")
Audio_Valid=os.path.join(EMOTIAUDIOPATH,"Val_audio.csv")



In [3]:
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct

class Audio_Data(Dataset):
    def __init__(self,csv_file,audio_path=None):
        self._table = pd.read_csv(csv_file,delimiter=' ')
        self._audio_table=pd.read_csv(audio_path)
    def __len__(self):
        return len(self._table)
    def __getitem__(self, idx):
        audio_feature=self._audio_table.loc[self._audio_table.file_name ==self._table.Vid_name[idx]+'.arff']
        audio_feature=pd.to_numeric(audio_feature.values[0][2:])
        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
        return (audio_feature,labels)
        



In [4]:
def train(num_epochs,name,model,train_dataloader,valid_dataloader,optimizer,criterion):
    f = open(name+".txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]
    model.train()
    for epochs in range(0,num_epochs):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (audio_file,label) in tqdm(enumerate(train_dataloader)):
         
            batch_size=audio_file.size(0)
            optimizer.zero_grad()
            audio_file=audio_file.float().to(device)
            output=model(audio_file)
            loss=criterion(output,label.to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            true_label=label.detach().cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
            if (i_batch+1)%84==0:
               # print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)

        torch.save(model.state_dict(),name+".pth")
        print("Validation\n")
        model.eval()   
        correct=0
        total_samples=0
        avg_vloss=0
        first=True

        for i_batch, (audio_file,label) in (enumerate(valid_dataloader)):
         
            batch_size=audio_file.size(0)
            optimizer.zero_grad()
            audio_file=audio_file.float().to(device)
            
            output=model(audio_file)
            loss=criterion(output,label.to(device))
            avg_vloss+=loss.item()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size

            true_label=label.detach().cpu()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
        print(conf_mat)
        avg_vloss=avg_vloss/len(valid_dataloader)
        print(avg_vloss)
        avg_vaccuracy=correct/(total_samples)
        print("Accuracy: ", avg_vaccuracy)

        data = " %f,%f,%f,%f \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)
        f.write(data)



    f.close()

In [5]:
model=video_model.AudioRecognition()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.
                              cuda.FloatTensor)
model.to(device)

AudioRecognition(
  (fc1): Linear(in_features=988, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [6]:
model(torch.rand(3,988))

tensor([[0.2450, 0.4796, 0.2753],
        [0.4802, 0.2599, 0.2599],
        [0.3152, 0.2651, 0.4197]], grad_fn=<SoftmaxBackward>)

In [7]:
train_set=Audio_Data(Train_label,Audio_Train)
valid_set=Audio_Data(Val_labels,Audio_Valid)


In [8]:
train_dataloader = DataLoader(train_set, batch_size=32
                       , num_workers=0,shuffle=True)

valid_dataloader = DataLoader(valid_set, batch_size=32
                       , num_workers=0,shuffle=False)

In [9]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.005, betas=(0.5, 0.999))
num_epochs=50
criterion = nn.CrossEntropyLoss()  

model_name="pre_embedded_audio"

In [10]:
train(num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion)

Training Epoch:  1 



Batch:  84 / 84
Batch Recognition loss:  0.981432318687439

[[327 271 208]
 [238 511 174]
 [358 283 291]]
Average_Loss:  1.0768274828082038
Average_Accuracy:  0.42427658774896654
Validation

[[ 38 119 142]
 [ 11 177  93]
 [ 18  90  78]]
1.116650549074014
Accuracy:  0.3825065274151436
Training Epoch:  2 



Batch:  84 / 84
Batch Recognition loss:  1.1166741847991943

[[198 304 304]
 [106 613 204]
 [159 280 493]]
Average_Loss:  1.0389756915115176
Average_Accuracy:  0.4900413378429162
Validation

[[ 96 176  27]
 [ 73 198  10]
 [ 81  78  27]]
1.0904515733321507
Accuracy:  0.41906005221932113
Training Epoch:  3 



Batch:  84 / 84
Batch Recognition loss:  1.1042845249176025

[[284 266 256]
 [166 579 178]
 [205 242 485]]
Average_Loss:  1.0168754302320027
Average_Accuracy:  0.506576475009395
Validation

[[ 46 141 112]
 [ 24 165  92]
 [ 21  48 117]]
1.0869081318378448
Accuracy:  0.4281984334203655
Training Epoch:  4 



Batch:  84 / 84
Batch Recognition loss:  1.1947815418243408

[[312 228 266]
 [152 544 227]
 [212 192 528]]
Average_Loss:  1.0126755656231017
Average_Accuracy:  0.5201052236001503
Validation

[[ 90  26 183]
 [ 61  56 164]
 [ 32  20 134]]
1.146715223789215
Accuracy:  0.36553524804177545
Training Epoch:  5 



...

Batch:  84 / 84
Batch Recognition loss:  0.9859333038330078

[[468 179 159]
 [178 626 119]
 [190 182 560]]
Average_Loss:  0.9139190238146555
Average_Accuracy:  0.6215708380308155
Validation

[[ 40 115 144]
 [ 35 163  83]
 [ 12  55 119]]
1.1115562270085018
Accuracy:  0.42036553524804177
Training Epoch:  47 



Batch:  84 / 84
Batch Recognition loss:  1.2052825689315796

[[463 168 175]
 [181 605 137]
 [190 149 593]]
Average_Loss:  0.9176195703801655
Average_Accuracy:  0.6242014280345735
Validation

[[106  70 123]
 [ 88 117  76]
 [ 40  28 118]]
1.0886364777882893
Accuracy:  0.4451697127937337
Training Epoch:  48 



Batch:  84 / 84
Batch Recognition loss:  1.1668592691421509

[[441 170 195]
 [165 610 148]
 [174 137 621]]
Average_Loss:  0.9138705340169725
Average_Accuracy:  0.6283352123261932
Validation

[[145 124  30]
 [100 160  21]
 [ 64  65  57]]
1.0673949991663296
Accuracy:  0.4725848563968668
Training Epoch:  49 



Batch:  84 / 84
Batch Recognition loss:  1.2483855485916138

[[435 168 203]
 [159 615 149]
 [174 153 605]]
Average_Loss:  0.9177954324654171
Average_Accuracy:  0.6219466366027809
Validation

[[123 107  69]
 [ 88 143  50]
 [ 45  55  86]]
1.0716205537319183
Accuracy:  0.4595300261096606
Training Epoch:  50 



Batch:  84 / 84
Batch Recognition loss:  1.1458749771118164

[[461 172 173]
 [173 606 144]
 [170 152 610]]
Average_Loss:  0.9074547035353524
Average_Accuracy:  0.6302142051860203
Validation

[[123  87  89]
 [ 88 137  56]
 [ 49  46  91]]
1.0774105166395505
Accuracy:  0.45822454308093996
